# Let's chat with a friend and I have a brain

Demo chat with Leolani that combine face recognition and storage in the BRAIN.

Don't forget to install emissor by `pip install .` at the root of this repo.
Install the requirements `pip install -r requirements.txt`
you might also have to run `python -m spacy download en`

Occasionally you have to kill the docker containers if you force close the chat.
`docker kill $(docker ps -q)`

In [1]:
import emissor as em
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import (
    Modality,
    ImageSignal,
    TextSignal,
    Mention,
    Annotation,
    Scenario,
)
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType


import uuid
from datetime import datetime
import cv2
import requests
import pickle

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util
import util.face_util as f_util

/Users/piek/Desktop/t-MA-Combots-2021/code/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
### Link your camera
camera = cv2.VideoCapture(0)

In [3]:
# Initialise the brain in GraphDB
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)

<class 'pathlib.PosixPath'>
2021-10-28 22:45:23,831 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Booted


2021-10-28 22:45:23.831 DEBUG basic_brain - __init__: Booted


2021-10-28 22:45:23,832 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Clearing brain


2021-10-28 22:45:23.832 DEBUG basic_brain - clear_brain: Clearing brain


2021-10-28 22:45:24,538 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Checking if ontology is in brain


...

2021-10-28 22:45:26.284 DEBUG basic_brain - __init__: Booted


2021-10-28 22:45:26,429 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query


2021-10-28 22:45:26.429 DEBUG basic_brain - _submit_query: Posting query


2021-10-28 22:45:26,436 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


2021-10-28 22:45:26.436 INFO trust_calculator - compute_trust_network: Computed trust for all known agents


## Standard initialisation of a scenario

In [4]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
agent = "Leolani2"
human = "Stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "./data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, agent)

Directory  ./data/2021-10-28-22:45:31  Created 
Directory  ./data/2021-10-28-22:45:31/image  Created 


### Loading the docker containers for face detection and face property detection

SKIP THIS IF DOCKER IMAGES ARE RUNNING

You only need to load the dockers once. The first time you load the docker, the images will be donwloaded from the DockerHub. This may take a few minutes depending on the speed of the internet connection. The images are cached in your local Docker installation.

One the images are in your local Docker, they are loaded instantaniously. Once the docker is started you do not need to start it again and you can skip the next commands.

In [3]:
container_fdr = f_util.start_docker_container("tae898/face-detection-recognition:v0.1", 10002)
container_ag = f_util.start_docker_container("tae898/age-gender:v0.2", 10003)

2021-10-28 09:29:12.348 INFO face_util - start_docker_container: starting a tae898/face-detection-recognition:v0.1 container ...
2021-10-28 09:29:18.268 INFO face_util - start_docker_container: starting a tae898/age-gender:v0.2 container ...


If there is a problem starting the dockers, you may need to kill them and start them again. Use the following command to kill and rerun the previous command. Note that if there are running already you should not restart. Starting it again gives an error that the port is occupied.

In [11]:
#!docker kill $(docker ps -q)

## We are now set to make a new friend

In [5]:
def process_text_and_think (scenario: Scenario, 
                  place_id:str, 
                  location: str, 
                  textSignal: TextSignal,
                  human: str,
                  my_brain:LongTermMemory):
    thoughts = ""
    chat = Chat(human)
    chat.add_utterance([UtteranceHypothesis(c_util.seq_to_text(textSignal.seq), 1.0)])
    chat.last_utterance.analyze()
    # No triple was extracted, so we missed three items (s, p, o)
    if chat.last_utterance.triple is None:
        utterance = "Any gossip" + '\n'
    else:
        triple = c_util.rephrase_triple_json_for_capsule(chat.last_utterance.triple)
        # A triple was extracted so we compare it elementwise
        capsule = c_util.scenario_utterance_and_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  textSignal, 
                                                                  human,
                                                                  chat.last_utterance.perspective, 
                                                                  triple)
        print('Capsule:', capsule)
        thoughts = my_brain.update(capsule, reason_types=True)
        #print(thoughts)
    return thoughts    

In [6]:
def process_new_friend_and_think (scenario: Scenario, 
                  place_id:str, 
                  location: str, 
                  human_id: str,
                  textSignal: TextSignal,
                  imageSignal: ImageSignal,
                  age: str,
                  gender: str,
                  human_name: str,
                  my_brain:LongTermMemory):
    age_thoughts = ""
    gender_thoughts = ""
    name_thoughts = ""


    if human_name:
        # A triple was extracted so we compare it elementwise
        perspective = {"certainty": 1, "polarity": 1, "sentiment": 1}
        capsule = c_util.scenario_utterance_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  textSignal,
                                                                  human_id,
                                                                  perspective,
                                                                  human_id,
                                                                  "label",
                                                                  human_name)

        name_thoughts = my_brain.update(capsule, reason_types=True)
        print('Name capsule:', capsule)


    if age:
        # A triple was extracted so we compare it elementwise
        capsule = c_util.scenario_image_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  imageSignal,
                                                                  "front_camera", 
                                                                  human_id,
                                                                  "age",
                                                                  str(age))

        age_thoughts = my_brain.update(capsule, reason_types=True)
        print('Age capsule:', capsule)

    if gender:
        capsule = c_util.scenario_image_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  imageSignal,
                                                                  "front_camera", 
                                                                  human_id,
                                                                  "gender",
                                                                  gender)

        gender_thoughts = my_brain.update(capsule, reason_types=True)
        print('Gender capsule:', capsule)

        #print(thoughts)
    return name_thoughts, age_thoughts, gender_thoughts    

In [7]:
def get_to_know_person(scenario: Scenario, agent:str, gender:str, age: str, uuid_name: str, embedding):
        ### This is a stranger
        ### We create the agent response and store it as a text signal
        human_name = "Stranger"
        response = (
            f"Hi there. We haven't met. I only know that \n"
            f"your estimated age is {age} \n and that your estimated gender is "
            f"{gender}. What's your name?"
        )
        print(f"{agent}: {response}")
        textSignal = d_util.create_text_signal(scenario, response)
        scenario.append_signal(textSignal)
        
        confirm = ""
        while confirm.lower().find("yes")==-1:
            ### We take the response from the user and store it as a text signal
            utterance = input("\n")
            textSignal = d_util.create_text_signal(scenario, utterance)
            scenario.append_signal(textSignal)
            print(utterance)
            #### We hack the response to find the name of our new fiend
            #### This name needs to be set in the scenario and assigned to the global variable human
            human_name = " ".join([foo.title() for foo in utterance.strip().split()])
            human_name = "_".join(human_name.split())
        
            response = (f"So your name is {human_name}?")
            print(f"{agent}: {response}")
            textSignal = d_util.create_text_signal(scenario, response)
            scenario.append_signal(textSignal)
            
            ### We take the response from the user and store it as a text signal
            confirm = input("\n")
            textSignal = d_util.create_text_signal(scenario, confirm)
            scenario.append_signal(textSignal)


        current_time = str(datetime.now().microsecond)
        human_id = human_name+"_t_"+current_time
        #### We create the embedding
        to_save = {"uuid": uuid_name["uuid"], "embedding": embedding}

        with open(f"./friend_embeddings/{human_id}.pkl", "wb") as stream:
            pickle.dump(to_save, stream)
            
        return human_id, human_name, textSignal

Friends of Leolani are saved in the embeddings folder

In [8]:
#First step is to identify the speaker

imagepath = ""
uuid_name = ""
# First signal
success, frame = camera.read()

if success:
    ### check if we know the human
    current_time = str(datetime.now().microsecond)
    imagepath = imagefolder + "/" + current_time + ".png"
    cv2.imwrite(imagepath, frame)
    imageSignal = d_util.create_image_signal(scenario, imagepath)
    (
        genders,
        ages,
        bboxes,
        faces_detected,
        det_scores,embeddings,
    ) = f_util.do_stuff_with_image(imagepath)

    # Here we assume that only one face is in the image
    # TODO: deal with multiple people.
    for k, (gender, age, bbox, uuid_name, faceprob, embedding) in enumerate(
        zip(genders, ages, bboxes, faces_detected, det_scores, embeddings)
    ):
        age = round(age["mean"])
        gender = "male" if gender["m"] > 0.5 else "female"
        bbox = [int(num) for num in bbox.tolist()]
    assert k == 0
    if uuid_name["name"] is None:
        ### This is a stranger
        

        human_id, human, textSignal = get_to_know_person(scenario, agent, gender, age, uuid_name, embedding)
                    
        name_thoughts, age_thoughts, gender_thoughts = process_new_friend_and_think (scenario, 
                  place_id, 
                  location, 
                  human_id,
                  textSignal,
                  imageSignal,
                  age,
                  gender,
                  human,
                  my_brain)
        
        ### The system responds to the processing of the new name input and stores it as a textsignal
        print(agent + f": Nice to meet you, {human}")
        response = f": Nice to meet you, {human}"
        textSignal = d_util.create_text_signal(scenario, response)
        scenario.append_signal(textSignal)

    else:
        ### We know this person
        human_id= uuid_name['name']
        human = human_id.split("_t_")[0]
        response = f"Hi {human}. Nice to see you again :)"
        print(f"{agent}: {response}")
        textSignal = d_util.create_text_signal(scenario, response)
        scenario.append_signal(textSignal)


2021-10-28 22:45:39.807 INFO face_util - load_binary_image: ./data/2021-10-28-22:45:31/image/762056.png image loaded!
2021-10-28 22:45:40.592 INFO face_util - run_face_api: got <Response [200]> from server!...
2021-10-28 22:45:40.593 INFO face_util - run_face_api: 1 faces deteced!
2021-10-28 22:45:40.595 INFO face_util - face_recognition: new face!
2021-10-28 22:45:40.664 INFO face_util - run_age_gender_api: got <Response [200]> from server!...


Leolani2: Hi there. We haven't met. I only know that 
your estimated age is 33 
 and that your estimated gender is male. What's your name?



 Piek


Piek
Leolani2: So your name is Piek?



 yes


...

2021-10-28 22:45:50,505 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query


2021-10-28 22:45:50.505 DEBUG basic_brain - _submit_query: Posting query


2021-10-28 22:45:50,510 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query


2021-10-28 22:45:50.510 DEBUG basic_brain - _submit_query: Posting query


Gender capsule: {'chat': '2021-10-28-22:45:31', 'turn': '0e5b4c78-abbc-45f9-bd73-522cc49c5c80', 'author': 'front-camera', 'utterance': '', 'position': 'image', 'perspective': <cltl.brain.infrastructure.api.Perspective object at 0x7f91203ac790>, 'subject': {'label': 'Piek_t_517960', 'type': 'person'}, 'predicate': {'type': 'gender'}, 'object': {'label': 'male', 'type': 'string'}, 'context_id': 'Leolani2', 'date': datetime.date(2021, 10, 28), 'place': 'Weesp', 'place_id': 64, 'country': 'NL', 'region': 'North Holland', 'city': 'Weesp', 'objects': [{'type': 'chair', 'confidence': 0.59, 'id': 1}, {'type': 'table', 'confidence': 0.73, 'id': 1}, {'type': 'pillbox', 'confidence': 0.32, 'id': 1}], 'people': [{'name': 'front_camera', 'confidence': 0.98, 'id': 1}], 'triple': piek-t-517960_gender_male [person_->_string]), 'type': <UtteranceType.STATEMENT: 0>}
Leolani2: Nice to meet you, Piek


## Proceed with the communication with an identified friend

In [9]:
### First prompt
response = "How are you doing"+human
textSignal = d_util.create_text_signal(scenario, response)
scenario.append_signal(textSignal)

print(agent + ": " + response)

utterance = input("\n")
print(human + ": " + utterance)

while not (utterance.lower() == "stop" or utterance.lower() == "bye"):
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    # @TODO: also annotate the textSignal
    # Apply some processing to the textSignal and add annotations
        
    # We process the utterance and store triples in the brain
    # We catch the thoughts as the response
    thoughts = process_text_and_think (scenario, 
                  place_id, 
                  location, 
                  textSignal, 
                  human, my_brain)
        
    ## We capture the image again
    if success:
        imageSignal = d_util.create_image_signal(scenario, imagepath)
        container_id = str(uuid.uuid4())

        #### Properties are now stored as annotations
        #### We do not store these proeprties again to the BRAIN
        for gender, age, bbox, name, faceprob in zip(
            genders, ages, bboxes, faces_detected, det_scores
        ):

            age = round(age["mean"])
            gender = "male" if gender["m"] > 0.5 else "female"
            bbox = [int(num) for num in bbox.tolist()]

            annotations = []

            annotations.append(
                {
                    "source": "machine",
                    "timestamp": current_time,
                    "type": "person",
                    "value": {
                        "name": name,
                        "age": age,
                        "gender": gender,
                        "faceprob": faceprob,
                    },
                }
            )

            mention_id = str(uuid.uuid4())
            segment = [
                {"bounds": bbox, "container_id": container_id, "type": "MultiIndex"}
            ]
            imageSignal.mentions.append(
                {"annotations": annotations, "id": mention_id, "segment": segment}
            )

        scenario.append_signal(imageSignal)

    # Create the response from the system and store this as a new signal
    # We could use the throughts to respond
    # @TODO generate a response from the thoughts

    utterance = "So you what do you want to talk about " + human + "\n"
    response = utterance[::-1]
    print(agent + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    # Getting the next input signals
    utterance = input("\n")

    success, frame = camera.read()
    if success:
        current_time = str(datetime.now().microsecond)
        imagepath = imagefolder + "/" + current_time + ".png"
        cv2.imwrite(imagepath, frame)
        (
            genders,
            ages,
            bboxes,
            faces_detected,
            det_scores,
            embeddings,
        ) = f_util.do_stuff_with_image(imagepath)


Leolani2: How are you doingPiek



 I like pizza


Piek: I like pizza
2021-10-28 22:45:59,231 -     INFO - cltl.triple_extraction.api.Chat (Piek)              000 - << Start of Chat with Piek >>


2021-10-28 22:45:59.231 INFO api - __init__: << Start of Chat with Piek >>


2021-10-28 22:45:59,232 -     INFO -               cltl.triple_extraction.api - Started POS tagger


...

2021-10-28 22:46:04,147 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query


2021-10-28 22:46:04.147 DEBUG basic_brain - _submit_query: Posting query


Leolani2: So you what do you want to talk about Piek




 Bye


2021-10-28 22:46:07.147 INFO face_util - load_binary_image: ./data/2021-10-28-22:45:31/image/102140.png image loaded!
2021-10-28 22:46:07.896 INFO face_util - run_face_api: got <Response [200]> from server!...
2021-10-28 22:46:07.898 INFO face_util - run_face_api: 1 faces deteced!
2021-10-28 22:46:07.942 INFO face_util - run_age_gender_api: got <Response [200]> from server!...


### Set the end time of the scenario, save it and stop the containers

After we stopped the interaction, we set the end time and save the scenario as EMISSOR data.

In [10]:
#scenario.scenario.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

In [16]:
### Stopping the docker containers
### This is only needed of you started them in this notebook

#f_util.kill_container(container_fdr)
#f_util.kill_container(container_ag)

In [11]:
#### Stop the camera when we are done
camera.release()

## End of notebook